In [1]:
# 라이브러리 설치 
# !pip install torch torchvision torchaudio

In [2]:
# 라이브러리 설치 
# !pip install torch torchvision torchaudio
# 라이브러리 로드 
import torch 
import torch.nn as nn
import torch.optim as optim

In [3]:
# 데이터셋을 생성 
# 독립, 종속 -> 파이토치에서는 독립, 종속 변수의 데이터의 타입은 Tensor 로 구성
# 독립 변수 -> 2차원 데이터
# 종속 변수 -> 2차원 데이터 (sklearn에서는 1차원)
X = torch.tensor( [ [1.0], [2.0], [3.0], [4.0] ] )  # 4행 1열
# 종속 변수는 독립 변수 데이터에서 2를 곱하고 1을 더해준 값으로 생성
Y = torch.tensor( [ [3.0], [5.0], [7.0], [9.0] ] )  # 4행 1열

In [4]:
type(X)

torch.Tensor

In [5]:
# 선형 회귀 
class LinearReg(nn.Module):
    # 생성자 함수 
    def __init__(self):
        # self: 클래스가 생성된 메모리의 주소
        # super(): 부모 클래스(nn.Module) <-- 상속
        super( LinearReg, self ).__init__()   # 부모 클래스의 생성자 함수를 실행
        # Linear()의 첫 번째 인자 - 입력 데이터(독립의 피쳐)의 크기
            # 두 번째 인자 - 출력 데이터(종석의 피쳐)의 크기
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)

In [6]:
# 모델을 생성 (회귀 모델)
# LinearReg 클래스에서, 생성자 함수 중 self를 제외하고 매개변수가 존재하지 않으므로
# 클래스 생성 시 넣어야 할 인자값 없음
model = LinearReg()

In [7]:
# 손실 함수
criterion = nn.MSELoss()
# 가중치 갱신 (경사 하강법)
# lr은 가중치를 변환할 때의 변환 비율
# 경사 하강법의 목적은 가중치를 천천히 내리며 최적의 조건을 찾는 것인데, 너무 크게 잡으면 그 구간을 지나갈 수도 있음.
# 너무 세세하게 잡으면 학습 시간이 너무 오래 걸림.
optimizer = optim.SGD(model.parameters(), lr= 0.01)

In [8]:
# 500번 반복 실행하면서 Loss를 확인
for epoch in range(500):
    # 순전파
    Y_pred = model(X)
    # 손실 함수
    loss = criterion(Y_pred, Y)
    # 역전파
    optimizer.zero_grad()       # 기울기 초기화
    loss.backward()             # 기울기 계산
    optimizer.step()            # 파라미터를 업데이트

    # 50회마다 loss의 값을 출력
    if (epoch + 1) % 50 == 0:
        print(f'Epoch: [{epoch+1}, 500], Loss: {round(loss.item(), 4)}')

Epoch: [50, 500], Loss: 0.0
Epoch: [100, 500], Loss: 0.0
Epoch: [150, 500], Loss: 0.0
Epoch: [200, 500], Loss: 0.0
Epoch: [250, 500], Loss: 0.0
Epoch: [300, 500], Loss: 0.0
Epoch: [350, 500], Loss: 0.0
Epoch: [400, 500], Loss: 0.0
Epoch: [450, 500], Loss: 0.0
Epoch: [500, 500], Loss: 0.0


In [9]:
# 예측값 확인
pred = model(X).detach()

for i in range(len(X)):
    print(f"독립: {X[i].item()}, 종속: {Y[i].item()}, 예측: {pred[i].item()}")

독립: 1.0, 종속: 3.0, 예측: 2.9989938735961914
독립: 2.0, 종속: 5.0, 예측: 4.999512195587158
독립: 3.0, 종속: 7.0, 예측: 7.000030517578125
독립: 4.0, 종속: 9.0, 예측: 9.00054931640625


In [10]:
# sklearn에서 제공하는 캘리포니아 집값 데이터를 이용하여 회귀 분석
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

In [11]:
# 데이터를 로드해 독립 종속 변수 데이터 생성
data = fetch_california_housing()

X = data['data']
Y = data['target']

print('독립 변수 데이터의 크기: ', X.shape)
print('종속 변수 데이터의 크기: ', Y.shape)

독립 변수 데이터의 크기:  (20640, 8)
종속 변수 데이터의 크기:  (20640,)


In [12]:
# 데이터를 학습, 평가 데이터로 분할
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state= 42
)

In [13]:
# StandardScaler를 이용하여 스케일링
# train 데이터를 기준으로 fitting (= train 데이터를 기준으로 범위를 지정)
    # 일반적으로 train, test 모두 같은 fit model에서 변환 작업해야 함
scaler= StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

In [14]:
# 총 4개의 데이터의 타입은 현재 array 형태 -> Tensor 형태로 변환
X_train_sc_tensor = torch.tensor( X_train_sc, dtype= torch.float32 )
X_test_sc_tensor = torch.tensor( X_test_sc, dtype= torch.float32 )
# 스케일링한 데이터와 하지 않은 데이터 비교
X_train_tensor = torch.tensor( X_train, dtype= torch.float32 )
X_test_tensor = torch.tensor( X_test, dtype= torch.float32 )

# 종속 변수 Y의 데이터는 현재 1차원 데이터 -> 2차원 데이터로 변환 -> Tensor 형태로 변환
Y_train_tensor = torch.tensor( Y_train.reshape(-1, 1), dtype= torch.float32 )
Y_test_tensor = torch.tensor( Y_test.reshape(-1, 1), dtype= torch.float32 )

In [15]:
# class 생성
class Reg(nn.Module):
    # 생성자 함수에서 self를 제외한 매개변수로 _dim을 지정
    # nn.linear()의 첫 번째 인자 - 입력값의 열의 크기
    def __init__(self, _dim):
        super(Reg, self).__init__()
        self.linear = nn.Linear(_dim, 1)
    
    def forward(self, x):
        return self.linear(x)

In [16]:
# 모델을 생성
# 학습 데이터의 피쳐의 개수
reg_model = Reg( X_train_sc.shape[1] )  # 8이 아닌 1인 이유: 

In [17]:
# 손실 함수
criterion = nn.MSELoss()
# 가중치 변환
optimizer = optim.SGD( reg_model.parameters(), lr= 0.01 )

In [18]:
# 학습 반복 실행
# 아까는 행 4개, 열 1개라 금방 끝났지만, 지금은 행 16512개, 열 8개라 시간이 더 걸리므로 300번만 실행
for epoch in range(300):
    # 순전파
    Y_pred = reg_model( X_train_sc_tensor )
    # 손실 함수
    loss = criterion( Y_pred, Y_train_tensor )
    # 기울기 초기화
    optimizer.zero_grad()
    # 역전파
    loss.backward()
    # 가중치 업데이트
    optimizer.step()
    # 30회마다 loss의 값을 출력해 변환 확인
    if (epoch + 1) % 30 == 0:
        print(f'Epoch: [{epoch+1}, 1000], Loss: {round(loss.item(), 4)}')

Epoch: [30, 1000], Loss: 1.9089
Epoch: [60, 1000], Loss: 1.0292
Epoch: [90, 1000], Loss: 0.7581
Epoch: [120, 1000], Loss: 0.6682
Epoch: [150, 1000], Loss: 0.6337
Epoch: [180, 1000], Loss: 0.6168
Epoch: [210, 1000], Loss: 0.6059
Epoch: [240, 1000], Loss: 0.5975
Epoch: [270, 1000], Loss: 0.5902
Epoch: [300, 1000], Loss: 0.5838


In [19]:
# 현재 모델의 학습 모드 -> 평가 모드로 변경 (학습 중지 X, 모드 변경 O)
# eval() 메서드: 평가 모드로 변경 (드롭아웃, 배치 정규화 등 학습 시에만 적용되는 기능들을 평가 시에 적용되지 않도록 변경)
# train() 메서드: 다시 학습 모드로 변경
reg_model.eval()

Reg(
  (linear): Linear(in_features=8, out_features=1, bias=True)
)

In [20]:
# 예측과 평가
# torch.no_grad(): 평가 시에 기울기(gradient) 계산을 하지 않도록 설정 -> 속도 증가해 메모리 절약
with torch.no_grad():
    Y_pred = reg_model( X_test_sc_tensor )  # test 데이터로 예측,
    test_loss = criterion( Y_pred, Y_test_tensor )  #  "   손실 함수 계산

In [21]:
# 성능 평가 지표
# MSE, RMSE 확인
rmse = np.sqrt(test_loss.item())
print('테스트 데이터의 MSE: ', round(test_loss.item(), 4))
print('테스트 데이터의 RMSE: ', round(rmse, 4))

테스트 데이터의 MSE:  0.5983
테스트 데이터의 RMSE:  0.7735


In [22]:
# 실제 값과 예측 값 비교
# 상위 10개만 출력해 확인
for i in range(10):
    print(f'실제 값: {Y_test[i].item()}, 예측 값: {round(Y_pred[i].item(), 3)}')

실제 값: 0.477, 예측 값: 0.963
실제 값: 0.458, 예측 값: 1.593
실제 값: 5.00001, 예측 값: 2.401
실제 값: 2.186, 예측 값: 2.723
실제 값: 2.78, 예측 값: 2.215
실제 값: 1.587, 예측 값: 2.135
실제 값: 1.982, 예측 값: 2.706
실제 값: 1.575, 예측 값: 2.177
실제 값: 3.4, 예측 값: 2.184
실제 값: 4.466, 예측 값: 4.116


---
연습
- 스탠다드 스케일링을 하지 않은 데이터로 학습하고 예측을 출력하시오

In [23]:
# 모델 생성
reg_model2 = Reg( X_train.shape[1] )
print(reg_model2)

Reg(
  (linear): Linear(in_features=8, out_features=1, bias=True)
)


In [24]:
# 손실 함수
criterion2 = nn.MSELoss()
# 가중치 변환
# Adam 옵티마이저: 적응적 학습률 기반 경사하강법
# 위처럼 SGD로 하면 NaN이 발생하는 경우가 있는데, Adam은 그런 경우가 거의 없음
optimizer2 = optim.Adam( reg_model2.parameters(), lr= 0.01 )

In [25]:
for epoch in range(300):
    # 순전파
    Y_pred2 = reg_model2( X_train_tensor )
    # 손실 함수
    loss2 = criterion2( Y_pred2, Y_train_tensor )
    # 기울기 초기화
    optimizer2.zero_grad()
    # 역전파
    loss2.backward()
    # 가중치 업데이트
    optimizer2.step()
    # 30회마다 loss2의 값을 출력해 변환 확인
    if (epoch + 1) % 30 == 0:
        print(f'Epoch: {epoch+1}, Loss: {round(loss2.item(), 4)}')

Epoch: 30, Loss: 423.9258
Epoch: 60, Loss: 248.1688
Epoch: 90, Loss: 155.2445
Epoch: 120, Loss: 101.3368
Epoch: 150, Loss: 65.0121
Epoch: 180, Loss: 42.7024
Epoch: 210, Loss: 30.033
Epoch: 240, Loss: 23.2073
Epoch: 270, Loss: 19.5866
Epoch: 300, Loss: 17.5729


In [26]:
reg_model2.eval()
with torch.no_grad():
    Y_pred2 = reg_model2(X_test_tensor)
    test_loss2 = criterion2(Y_pred2, Y_test_tensor).item()

In [27]:
# 실제 값과 예측 값 비교
for i in range(3):
    print(f'실제 값: {Y_test[i].item()}, 예측 값: {round(Y_pred2[i].item(), 4)}')

실제 값: 0.477, 예측 값: 0.7937
실제 값: 0.458, 예측 값: 2.5152
실제 값: 5.00001, 예측 값: 9.9949


In [28]:
# mse 보는 게 나을 것 같음
rmse2 = np.sqrt(test_loss2)
print('테스트 데이터의 MSE (스케일링 X): ', round(test_loss2, 4))
print('테스트 데이터의 RMSE (스케일링 X): ', round(rmse2, 4))

테스트 데이터의 MSE (스케일링 X):  17.1117
테스트 데이터의 RMSE (스케일링 X):  4.1366


---
### 파이토치를 이용한 분류 분석

In [29]:
# 라이브러리 로드
# iris 데이터를 이용
from sklearn.datasets import load_iris
# 분류 검증 정확도
from sklearn.metrics import accuracy_score

In [42]:
iris = load_iris()
X = iris['data']
Y = iris['target']

In [43]:
# train, test 데이터 분할
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state= 42
)

In [44]:
# 스탠다드 스케일러를 이용한 스케일링
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)   # 위의 연습처럼 스케일링하지 않은 데이터를 쓰지 않음
X_test = scaler.transform(X_test)

In [45]:
# 분류 모델에서 Tensor 데이터로 변환
# 독립 변수 -> float32 타입으로 변경
X_train_tensor = torch.tensor(X_train, dtype= torch.float32)
X_test_tensor = torch.tensor(X_test, dtype= torch.float32)
# 종속 변수 -> long 타입으로 변경 (분류 모델에서는 정수 형태여야 하므로)
Y_train_tensor = torch.tensor(Y_train, dtype= torch.long)
Y_test_tensor = torch.tensor(Y_test, dtype= torch.long)

In [46]:
Y_train_tensor
# 0, 1, 2로 구성된 정수 형태임 -> 세 가지 품종을 분류하는 문제임

tensor([0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0, 0, 1, 2, 2, 1, 2, 1, 2, 1, 0,
        2, 1, 0, 0, 0, 1, 2, 0, 0, 0, 1, 0, 1, 2, 0, 1, 2, 0, 2, 2, 1, 1, 2, 1,
        0, 1, 2, 0, 0, 1, 1, 0, 2, 0, 0, 1, 1, 2, 1, 2, 2, 1, 0, 0, 2, 2, 0, 0,
        0, 1, 2, 0, 2, 2, 0, 1, 1, 2, 1, 2, 0, 2, 1, 2, 1, 1, 1, 0, 1, 1, 0, 1,
        2, 2, 0, 1, 2, 2, 0, 2, 0, 1, 2, 2, 1, 2, 1, 1, 2, 2, 0, 1, 2, 0, 1, 2])

In [47]:
X.shape
# (150, 4) -> 피쳐가 4개

(150, 4)

In [48]:
# 모델 정의
class IrisClass(nn.Module):
    def __init__(self):
        # 부모 클래스의 생성자 함수 실행
        super(IrisClass, self).__init__()
        # 분류 모델 선택
        self.model = nn.Sequential(
            # Linear(열의 개수, 분류 클래스의 개수(이 경우 0,1,2 세 가지))
            nn.Linear(4, 3)
        )
        
    def forward(self, x):
        return self.model(x)

In [49]:
# 모델 생성
cls_model = IrisClass()

In [50]:
# 손실 함수 / 옵티마이저 생성
# 분류 모델에서의 손실 함수
criterion = nn.CrossEntropyLoss()
# 옵티마이저
optimizer = optim.SGD(cls_model.parameters(), lr= 0.01)

In [51]:
# 반복 학습 시행
for epoch in range(300):
    # 순전파
    Y_pred = cls_model(X_train_tensor)
    # 손실 함수
    loss = criterion(Y_pred, Y_train_tensor)
    # 역전파
    optimizer.zero_grad()       # 기울기 초기화
    loss.backward()             # 기울기 계산
    optimizer.step()            # 파라미터를 업데이트

    # 30회마다 loss의 값을 출력
    if (epoch + 1) % 30 == 0:
        print(f'Epoch: {epoch+1}, Loss: {round(loss.item(), 4)}')

Epoch: 30, Loss: 0.9652
Epoch: 60, Loss: 0.7877
Epoch: 90, Loss: 0.6768
Epoch: 120, Loss: 0.6056
Epoch: 150, Loss: 0.5573
Epoch: 180, Loss: 0.5229
Epoch: 210, Loss: 0.497
Epoch: 240, Loss: 0.4767
Epoch: 270, Loss: 0.4604
Epoch: 300, Loss: 0.4468


In [52]:
# 평가 - 실제 값과 예측 값 비교
# 정확도
cls_model.eval()
with torch.no_grad():
    pred = cls_model(X_test_tensor)
    # torch.argmax(): 텐서에서 최댓값이 위치하는 인덱스만 반환
    # torch.max(): 특정 차원에서 최댓값 자체와 그 값이 위치하는 인덱스를 함께 반환
    pred_idx = torch.argmax(pred, axis=1)  # 예측 값에서 가장 큰 값의 인덱스를 반환
    # 정확도를 계산
    acc = accuracy_score(Y_test, pred_idx)
print('분류 모델의 정확도: ', acc)

분류 모델의 정확도:  0.9333333333333333


In [53]:
pred
# 열이 3개 -> 3개의 클래스에 대한 예측 엔트로피 계수를 의미
# 세 열 중 가장 높은 값을 예측한 클래스로 판단
# 0번째 열: 클래스 0에 대한 예측 확률
# 1번째 열: 클래스 1에 대한 예측 확률
# 2번째 열: 클래스 2에 대한 예측 확률

tensor([[-1.0025,  1.0266,  0.6385],
        [ 1.2278, -0.6373, -1.3158],
        [-2.9213,  2.3584,  3.3738],
        [-0.8246,  0.7221,  0.7899],
        [-1.5411,  1.4088,  1.3517],
        [ 0.9200, -0.4953, -1.4522],
        [-0.3705,  0.3331,  0.0942],
        [-1.4249,  0.9487,  2.3142],
        [-1.8894,  1.5472,  0.9929],
        [-0.8022,  0.7578,  0.2349],
        [-0.9598,  0.7029,  1.7384],
        [ 0.8784, -0.3826, -2.1532],
        [ 1.0504, -0.5183, -1.6257],
        [ 0.9256, -0.3993, -2.0655],
        [ 1.6758, -1.0469, -1.7948],
        [-0.5534,  0.5159,  1.1135],
        [-1.3695,  1.0044,  2.1154],
        [-0.9094,  0.8703,  0.0151],
        [-0.7118,  0.7057,  0.3964],
        [-1.5335,  1.0981,  2.0115],
        [ 1.1631, -0.6294, -2.0990],
        [-0.8788,  0.6796,  1.2325],
        [ 1.1825, -0.7135, -1.7140],
        [-1.5152,  1.1303,  1.9179],
        [-1.3409,  1.2510,  3.0104],
        [-1.4337,  0.9520,  2.2042],
        [-2.0966,  1.7568,  1.9268],
 